# Limpieza de datos



In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
plt.rcParams['figure.figsize'] = (10, 4)

In [25]:
# 1) Carga
DATA_PATH = 'EDA_data_airbnb_madrid.csv'

df_raw = pd.read_csv(DATA_PATH)
print('Shape:', df_raw.shape)
df_raw.head()

Shape: (15036, 30)


,id,name,host_id,host_response_time,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,30320,Apartamentos Dana Sol,130907,within an hour,True,True,Sol,Entire rental unit,Entire home/apt,2,1.0,1.0,2.0,$157.00,5,50,5.0,50.0,16,46,76,173,1,1,4.71,4.88,4.82,4.78,4.90,4.69
1,40916,Apartasol Apartamentos Dana,130907,within an hour,True,True,Universidad,Entire rental unit,Entire home/apt,2,1.0,1.0,3.0,$143.00,5,50,5.0,50.0,10,40,66,53,4,1,4.71,4.90,4.87,4.81,4.88,4.59
2,62423,MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID,303845,within an hour,True,True,Justicia,Private room in rental unit,Private room,4,1.5,1.0,2.0,$65.00,1,30,1.0,30.0,4,16,34,249,41,4,4.78,4.46,4.80,4.86,4.97,4.60
3,70073,Adorable Apartment Malasaña-Gran Via,353738,within an hour,True,True,Universidad,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,$116.00,30,360,30.0,1125.0,3,3,30,36,1,0,4.49,4.42,4.43,4.40,4.60,4.09
4,70310,"Heart of Malasaña Cozy, Quiet & Sunny Apartment",353738,within an hour,True,True,Universidad,Entire rental unit,Entire home/apt,2,1.0,1.0,2.0,$79.00,30,360,28.6,1125.0,0,12,42,31,2,0,4.65,4.55,4.80,4.71,4.87,4.10


## 2) Limpieza base

- "price": a numérico
- duplicados
- tipos categóricos
- checks rápidos de nulos

In [26]:
# Copia de trabajo

df = df_raw.copy()

# Precio a float
# Soporta formatos tipo '€1,234.00', '$123', '123.45' pero mejor quitar los simbolitos

def to_price(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    s = s.replace('€','').replace('$','').strip()
    # quita separadores de miles comunes
    s = s.replace(',', '')
    try:
        return float(s)
    except:
        return np.nan

# Si ya es numérico, lo deja igual
if df['price'].dtype == 'object':
    df['price'] = df['price'].apply(to_price)

# Nulos en price
na_price = df['price'].isna().sum()
print('Nulos en price:', na_price)

# Mejor eliminar filas con price nulo (si son muy pocas)
if na_price > 0:
    df = df.dropna(subset=['price'])

# Duplicados
dups = df.duplicated().sum()
print('Duplicados:', dups)
if dups:
    df = df.drop_duplicates()

df.info()

Nulos en price: 11
Duplicados: 0
<class 'pandas.core.frame.DataFrame'>
Index: 15025 entries, 0 to 15035
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           15025 non-null  int64  
 1   name                         15025 non-null  object 
 2   host_id                      15025 non-null  int64  
 3   host_response_time           15025 non-null  object 
 4   host_has_profile_pic         15025 non-null  bool   
 5   host_identity_verified       15025 non-null  bool   
 6   neighbourhood_cleansed       15025 non-null  object 
 7   property_type                15025 non-null  object 
 8   room_type                    15025 non-null  object 
 9   accommodates                 15025 non-null  int64  
 10  bathrooms                    15025 non-null  float64
 11  bedrooms                     15025 non-null  float64
 12  beds                         15025 non-null  f

In [27]:
# Nulos por columna (top)
na = df.isna().sum().sort_values(ascending=False)
na[na>0].head(20)

Series([], dtype: int64)

## 3) Tipos y variables derivadas (según recomendaciones)

- Convertir variables a categoría para gráficos y groupby.
- "host_response_time" como categoría *ordenada*.
- Reviews: "review_score_mean" y "review_tier" (escala).

In [28]:
# 3.1) Categorías
cat_cols = [
    'host_response_time',
    'neighbourhood_cleansed',
    'property_type',
    'room_type'
]

for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype('category')

# Booleanos (por si vinieran como object)
bool_cols = ['host_has_profile_pic', 'host_identity_verified']
for c in bool_cols:
    if c in df.columns and df[c].dtype == 'object':
        df[c] = df[c].astype('bool')

df.dtypes

id                                int64
name                             object
host_id                           int64
host_response_time             category
host_has_profile_pic               bool
host_identity_verified             bool
neighbourhood_cleansed         category
property_type                  category
room_type                      category
accommodates                      int64
bathrooms                       float64
bedrooms                        float64
beds                            float64
price                           float64
minimum_nights                    int64
maximum_nights                    int64
minimum_nights_avg_ntm          float64
maximum_nights_avg_ntm          float64
availability_30                   int64
availability_60                   int64
availability_90                   int64
number_of_reviews                 int64
number_of_reviews_ltm             int64
number_of_reviews_l30d            int64
review_scores_accuracy          float64


In [29]:
# 3.2) host_response_time ordenado (si existe) "Es basicamente ordenarlo de mas rapido a mas lento"
if 'host_response_time' in df.columns:
    order = [
        'within an hour',
        'within a few hours',
        'within a day',
        'a few days or more'
    ]
    # Mantiene solo las categorías presentes
    present = [x for x in order if x in df['host_response_time'].cat.categories]
    # Añade las que existan pero no estén en el orden, al final
    extras = [x for x in df['host_response_time'].cat.categories if x not in present]
    ordered_cats = present + extras
    df['host_response_time'] = df['host_response_time'].cat.set_categories(ordered_cats, ordered=True)

df['host_response_time'].value_counts(dropna=False).head(10)

host_response_time
within an hour        11746
within a few hours     1688
within a day           1143
a few days or more      448
Name: count, dtype: int64

In [30]:
# 3.3) Reviews: media y tier
review_cols = [
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

present_reviews = [c for c in review_cols if c in df.columns] # Solo las que existen en nuestro dataset

df['review_score_mean'] = df[present_reviews].mean(axis=1) # La media me valdrá para usarla de "indicador único"

# Escala (La podeis cambiar si quereis pero me parecia coherente algo asi no se)
# Nota: suele haber muchos valores 4.6-4.9
bins = [-np.inf, 4.0, 4.5, 4.8, np.inf]
labels = ['Baja (≤4.0)', 'Media (4.0–4.5]', 'Alta (4.5–4.8]', 'Excelente (>4.8)']

df['review_tier'] = pd.cut(df['review_score_mean'], bins=bins, labels=labels)
df['review_tier'] = df['review_tier'].astype('category') # Mejor como categoría para análisis posteriores (gráficas etc)

df[['review_score_mean', 'review_tier']].head()

,review_score_mean,review_tier
0,4.796667,Alta (4.5–4.8]
1,4.793333,Alta (4.5–4.8]
2,4.745000,Alta (4.5–4.8]
3,4.405000,Media (4.0–4.5]
4,4.613333,Alta (4.5–4.8]


## 4) Dataset final (opcional: columnas a eliminar)

Aquí yo he quitado las primeras columnas que no las necesitamos para nada



In [31]:
# Columnas recomendadas para el EDA (sin IDs ni nombre) ((no necesarias para el análisis exploratorio))
cols_to_drop = ['id', 'host_id', 'name']
cols_to_drop = [c for c in cols_to_drop if c in df.columns]

df_eda = df.drop(columns=cols_to_drop)
print('df_eda shape:', df_eda.shape)
df_eda.head()

df_eda shape: (15025, 29)


,host_response_time,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,review_score_mean,review_tier
0,within an hour,True,True,Sol,Entire rental unit,Entire home/apt,2,1.0,1.0,2.0,157.0,5,50,5.0,50.0,16,46,76,173,1,1,4.71,4.88,4.82,4.78,4.90,4.69,4.796667,Alta (4.5–4.8]
1,within an hour,True,True,Universidad,Entire rental unit,Entire home/apt,2,1.0,1.0,3.0,143.0,5,50,5.0,50.0,10,40,66,53,4,1,4.71,4.90,4.87,4.81,4.88,4.59,4.793333,Alta (4.5–4.8]
2,within an hour,True,True,Justicia,Private room in rental unit,Private room,4,1.5,1.0,2.0,65.0,1,30,1.0,30.0,4,16,34,249,41,4,4.78,4.46,4.80,4.86,4.97,4.60,4.745000,Alta (4.5–4.8]
3,within an hour,True,True,Universidad,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,116.0,30,360,30.0,1125.0,3,3,30,36,1,0,4.49,4.42,4.43,4.40,4.60,4.09,4.405000,Media (4.0–4.5]
4,within an hour,True,True,Universidad,Entire rental unit,Entire home/apt,2,1.0,1.0,2.0,79.0,30,360,28.6,1125.0,0,12,42,31,2,0,4.65,4.55,4.80,4.71,4.87,4.10,4.613333,Alta (4.5–4.8]


In [ ]:
# Guardar dataset limpio 
OUT_CSV = 'airbnb_madrid_clean_final.csv'
df_eda.to_csv(OUT_CSV, index=False)
print('Guardado:', OUT_CSV)

Guardado: airbnb_madrid_clean_final.csv
